In [1]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from tqdm import tqdm
import json
import click
import os
import numpy as np
from datasets import load_dataset

def build_model_and_tokenizer(model_name, adapter_name=None, device: str = 'cuda'):
    # 1) Load tokenizer & model
    tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
    model = (
        AutoModelForCausalLM
        .from_pretrained(model_name, trust_remote_code=True)
        .to(device)
    )
    if adapter_name is not None:
        model.load_adapter(adapter_name)
    model.eval()

    # 2) (Optional) Compile for speed if you're on PyTorch 2.x
    if torch.backends.cuda.is_built():
        try:
            model = torch.compile(model)
        except Exception:
            pass

    return model, tokenizer

In [2]:
dset = load_dataset("Salesforce/xlam-function-calling-60k", split='train')
dset[0]

{'id': 0,
 'query': 'Where can I find live giveaways for beta access and games?',
 'answers': '[{"name": "live_giveaways_by_type", "arguments": {"type": "beta"}}, {"name": "live_giveaways_by_type", "arguments": {"type": "game"}}]',
 'tools': '[{"name": "live_giveaways_by_type", "description": "Retrieve live giveaways from the GamerPower API based on the specified type.", "parameters": {"type": {"description": "The type of giveaways to retrieve (e.g., game, loot, beta).", "type": "str", "default": "game"}}}]'}

In [3]:
model, tokenizer = build_model_and_tokenizer('unsloth/Qwen3-4B-unsloth-bnb-4bit')

In [4]:
i = 0
q = dset[i]['query']
# inputs = format_question_qwen(questions, tokenizer)
# batch_size = len(questions)

# # Generate k samples in parallel
# with torch.no_grad():
#     out_ids = model.generate(
#         input_ids=inputs["input_ids"],
#         attention_mask=inputs["attention_mask"],
#         do_sample=True,
#         temperature=temperature,
#         top_p=top_p,
#         max_new_tokens=max_new_tokens,
#         pad_token_id=tokenizer.eos_token_id,
#         num_return_sequences=k,
#         use_cache=True
#     )
print(q)

Where can I find live giveaways for beta access and games?


In [18]:
# Please use our provided instruction prompt for best performance
task_instruction = """
You are an expert in composing functions. You are given a question and a set of possible functions. 
Based on the question, you will need to make one or more function/tool calls to achieve the purpose. 
If none of the functions can be used, point it out and refuse to answer. 
If the given question lacks the parameters required by the function, also point it out.
""".strip()

format_instruction = """
Put your final answer in \\boxed{}. The final answer MUST strictly adhere to the following JSON format, and NO other text MUST be included.
The example format is as follows. Please make sure the parameter type is correct. If no function call is needed, please make tool_calls an empty list '[]'.
\\boxed{
[
    {"name": "TOOL_NAME_1, "arguments" : {"PARAMETER_NAME_1" : "VALUE_1", .....}},
    ....(more tool calls as required)
]
}
""".strip()

# Helper function to build the input prompt for our model
def build_prompt(tools: list, query: str):
    prompt = f"[BEGIN OF TASK INSTRUCTION]\n{task_instruction}\n[END OF TASK INSTRUCTION]\n\n"
    prompt += f"[BEGIN OF AVAILABLE TOOLS]\n{tools}\n[END OF AVAILABLE TOOLS]\n\n"
    prompt += f"[BEGIN OF FORMAT INSTRUCTION]\n{format_instruction}\n[END OF FORMAT INSTRUCTION]\n\n"
    prompt += f"[BEGIN OF QUERY]\n{query}\n[END OF QUERY]\n\n"
    return prompt

# [{"name": "live_giveaways_by_type", "arguments": {"type": "beta"}}, {"name": "live_giveaways_by_type", "arguments": {"type": "game"}}]

i = 0
q = dset[i]['query']
answer = dset[i]['answers']
prompt = build_prompt(dset[i]['tools'], q)
print(prompt)

[BEGIN OF TASK INSTRUCTION]
You are an expert in composing functions. You are given a question and a set of possible functions. 
Based on the question, you will need to make one or more function/tool calls to achieve the purpose. 
If none of the functions can be used, point it out and refuse to answer. 
If the given question lacks the parameters required by the function, also point it out.
[END OF TASK INSTRUCTION]

[BEGIN OF AVAILABLE TOOLS]
[{"name": "live_giveaways_by_type", "description": "Retrieve live giveaways from the GamerPower API based on the specified type.", "parameters": {"type": {"description": "The type of giveaways to retrieve (e.g., game, loot, beta).", "type": "str", "default": "game"}}}]
[END OF AVAILABLE TOOLS]

[BEGIN OF FORMAT INSTRUCTION]
Put your final answer in \boxed{}. The final answer MUST strictly adhere to the following JSON format, and NO other text MUST be included.
The example format is as follows. Please make sure the parameter type is correct. If no 

In [19]:
formatted_prompt = tokenizer.apply_chat_template(
    [{'role': 'user', 'content': prompt}],
    tokenize=False, add_generation_prompt=True, enable_thinking=False
)

inputs = tokenizer(formatted_prompt, return_tensors="pt", padding=True, truncation=True).to('cuda')


# Generate k samples in parallel
with torch.no_grad():
    out_ids = model.generate(
        input_ids=inputs["input_ids"],
        attention_mask=inputs["attention_mask"],
        do_sample=True,
        pad_token_id=tokenizer.eos_token_id,
        use_cache=True
    )
decoded = tokenizer.batch_decode(out_ids[:, len(inputs.input_ids[0]):], skip_special_tokens=True)[0]
print(decoded)

\boxed{
[
    {"name": "live_giveaways_by_type", "arguments" : {"type" : "beta"}},
    {"name": "live_giveaways_by_type", "arguments" : {"type" : "game"}}
]
}


In [14]:
print(answer)

[{"name": "live_giveaways_by_type", "arguments": {"type": "beta"}}, {"name": "live_giveaways_by_type", "arguments": {"type": "game"}}]
